# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import gspread

## Data read-in

In [2]:
df = pd.read_csv('Prager map details - final.csv')

In [3]:
# # Drop the 'URL' column if you no longer need it
# df = df.drop(columns=['URL'])

In [4]:
# df['Description'].iloc[0]

## Geocode

In [5]:
df.columns

Index(['#', 'Street', 'City', 'County', 'State', 'Lawsuit/foreclosure',
       'Landlord Lender', 'A-1', 'A-2', 'B', 'Total Landlord loan',
       'Ground Lease Lender', 'A-1.1', 'A-2.1', 'B.1', 'GL Lender 2',
       'Unnamed: 16', 'Total Ground Lease loan', 'TOTAL LOAN'],
      dtype='object')

In [6]:
df['Full_Address'] = df['#'] + ' ' + df['Street'] + ' ' + df['City'] + ' ' + df['County'] + ' ' + df['State']

In [7]:
df['Display_Address'] = df['#'] + ' ' + df['Street'] + ' ' + df['City'] + ',' + df['State']

In [8]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [9]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,#,Street,City,County,State,Lawsuit/foreclosure,Landlord Lender,A-1,A-2,B,...,GL Lender 2,Unnamed: 16,Total Ground Lease loan,TOTAL LOAN,Full_Address,Display_Address,geocoded,lat,lon,geometry
0,333,City Blvd W,Orange County,Orange,CA,NaN,Technology FCU,75.750,NaN,NaN,...,NaN,NaN,0.000,75.750,333 City Blvd W Orange County Orange CA,"333 City Blvd W Orange County,CA","(33.7870982, -117.8936947)",33.787098,-117.893695,POINT (-117.89369 33.78710)
1,"300, 400, 801, 901",Int'l Pwy (+ 1001 Heathrow Park Lane),Lake Mary,Seminole,FL,Y,UMB,70.000,NaN,18.000,...,NaN,NaN,79.700,167.700,"300, 400, 801, 901 Int'l Pwy (+ 1001 Heathrow ...","300, 400, 801, 901 Int'l Pwy (+ 1001 Heathrow ...","(28.7801958, -81.356084)",28.780196,-81.356084,POINT (-81.35608 28.78020)
3,263,Shuman Blvd,Naperville,DuPage,IL,NaN,UMB,27.000,NaN,8.500,...,NaN,NaN,45.430,80.930,263 Shuman Blvd Naperville DuPage IL,"263 Shuman Blvd Naperville,IL","(41.8043611, -88.1514825)",41.804361,-88.151483,POINT (-88.15148 41.80436)
5,3025,Highland,Downers Grove,DuPage,IL,NaN,UMB,23.000,NaN,5.000,...,NaN,NaN,32.250,60.250,3025 Highland Downers Grove DuPage IL,"3025 Highland Downers Grove,IL","(41.8330351, -88.0056721)",41.833035,-88.005672,POINT (-88.00567 41.83304)
6,35,W Wacker Dr,Chicago,Cooks,IL,NaN,UMB,171.500,NaN,45.500,...,NaN,NaN,220.000,437.000,35 W Wacker Dr Chicago Cooks IL,"35 W Wacker Dr Chicago,IL","(41.8857787, -87.6294925)",41.885779,-87.629492,POINT (-87.62949 41.88578)
12,200,S College St,Charlotte,Mecklenburg,NC,NaN,UMB,50.000,NaN,15.262,...,NaN,NaN,60.000,125.262,200 S College St Charlotte Mecklenburg NC,"200 S College St Charlotte,NC","(35.22561, -80.8434381)",35.225610,-80.843438,POINT (-80.84344 35.22561)
13,801,Cherry St,Fort Worth,Tarrant,TX,Y,UMB,68.000,NaN,18.000,...,NaN,NaN,82.858,168.858,801 Cherry St Fort Worth Tarrant TX,"801 Cherry St Fort Worth,TX","(32.7503245, -97.33453929999999)",32.750324,-97.334539,POINT (-97.33454 32.75032)
15,20001,Euclid Ave,Euclid,Cuyahoga,OH,Y,Wells Fargo,24.000,NaN,NaN,...,NaN,NaN,0.000,24.000,20001 Euclid Ave Euclid Cuyahoga OH,"20001 Euclid Ave Euclid,OH","(41.5656431, -81.5425015)",41.565643,-81.542502,POINT (-81.54250 41.56564)
16,300,E John Carpenter Fwy,Irving,Dallas,TX,NaN,UMB,37.000,NaN,11.500,...,NaN,NaN,55.000,103.500,300 E John Carpenter Fwy Irving Dallas TX,"300 E John Carpenter Fwy Irving,TX","(32.8626749, -96.93971069999999)",32.862675,-96.939711,POINT (-96.93971 32.86267)
21,"1100, 1150, 1200",American Blvd,Hopewell,Mercer,NJ,NaN,TD BANK,60.000,NaN,NaN,...,NaN,NaN,44.300,104.300,"1100, 1150, 1200 American Blvd Hopewell Mercer NJ","1100, 1150, 1200 American Blvd Hopewell,NJ","(40.3029512, -74.8059905)",40.302951,-74.805990,POINT (-74.80599 40.30295)


In [10]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [11]:
gdf.to_file('data.geojson', driver='GeoJSON')

In [12]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Prager_Map_10_30_24


In [13]:
df.columns

Index(['#', 'Street', 'City', 'County', 'State', 'Lawsuit/foreclosure',
       'Landlord Lender', 'A-1', 'A-2', 'B', 'Total Landlord loan',
       'Ground Lease Lender', 'A-1.1', 'A-2.1', 'B.1', 'GL Lender 2',
       'Unnamed: 16', 'Total Ground Lease loan', 'TOTAL LOAN', 'Full_Address',
       'Display_Address', 'geocoded', 'lat', 'lon', 'geometry'],
      dtype='object')

<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>

In [14]:
gdf.columns

Index(['#', 'Street', 'City', 'County', 'State', 'Lawsuit/foreclosure',
       'Landlord Lender', 'A-1', 'A-2', 'B', 'Total Landlord loan',
       'Ground Lease Lender', 'A-1.1', 'A-2.1', 'B.1', 'GL Lender 2',
       'Unnamed: 16', 'Total Ground Lease loan', 'TOTAL LOAN', 'Full_Address',
       'Display_Address', 'geocoded', 'lat', 'lon', 'geometry'],
      dtype='object')